In [223]:
import pandas as pd
import numpy as np

file_path = '2020_LER_20200804_V006.xlsx'
datatype = 'n'
num_input=4
num_output=6
num_in_cycle=50
num_of_cycle=200
num_total = num_in_cycle * num_of_cycle
num_train=75
num_val=10
num_test=15
x_cols="B:G"
y_cols="H:P"
header=0


data_x = pd.read_excel('./'+file_path, sheet_name='Generated DATAs',usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('./'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

X_all_temp , Y_all_temp = np.zeros((num_total, num_input+1)), np.zeros((num_total, num_output)) # onehot encoding
X_per_cycle_temp, Y_per_cycle_temp = np.zeros((num_of_cycle, num_input+1)), np.zeros((num_of_cycle, num_output))

# DATA_X preprocessing

# 1. N, P to 10, 01 (one-hot encoding)
data_x =pd.get_dummies(data_x, columns=['PNMOS'])

# 2. Remove unrequired column ( Wfin [nm], alpha )

# X: Wfin, alpha
data_x = data_x.drop('Wfin [nm]', axis=1)
data_x = data_x.drop('alpha', axis=1)

# Y: 
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)

# X_per_cycle
for i in range(num_of_cycle):    
    #print(data_x[i*num_in_cycle:i*num_in_cycle+1])
    X_per_cycle_temp[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values

# X_all
X_all_temp = np.repeat(X_per_cycle,num_in_cycle,axis=0)

# DATA_Y preprcoessing

# Y_all
for i in range(num_total):
    Y_all_temp[i] = data_y[i:i+1].values
    
# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle_temp[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

X_all = np.empty(num_input+1)
Y_all = np.empty(num_output)
         
for i in range(num_of_cycle):
    X_all = np.vstack((X_all, X_all_temp[num_in_cycle*(2*i):num_in_cycle*(2*i)+num_in_cycle]))
    X_per_cycle = X_per_cycle_temp[::2]
    Y_all = np.vstack((Y_all, Y_all_temp[num_in_cycle*(2*i):num_in_cycle*(2*i)+num_in_cycle]))
    Y_per_cycle = Y_per_cycle_temp[::2]
            
X_all = X_all[1:]
Y_all = Y_all[1:]       
    
print(X_all.shape, Y_all.shape, X_per_cycle.shape, Y_per_cycle.shape)   
    
y_mean_cov = mean_cov(Y_all, num_in_cycle, num_of_cycle//2, num_output)
# print(y_mean_cov)

norm_x, norm_y, y_mean, y_std = normalize(X_per_cycle, y_mean_cov)

    


(2500, 5) (5000, 6) (100, 5) (100, 6)
Y_all size:  (5000, 6)
(100, 5) (100, 27)
[ 2.25020000e-12  4.29014000e-05  1.66996000e-05  3.40960000e-01
  3.48600000e-01  6.16506000e+01  8.57633878e-26  3.24849020e-13
  1.11934531e-13  6.85551020e-06  4.85714286e-06  1.72450204e-02
  1.02302163e-19  8.12580408e-20  1.41867918e-13 -7.57297959e-16
 -9.34432653e-10 -7.22873469e-10 -6.33612245e-16 -7.68000000e-10
 -6.09755102e-10  5.67755102e-06  3.73673673e-14  4.47430204e-08
  3.70135102e-08 -3.24077551e-04 -2.73612245e-04]
[ 2.26560000e-12  3.95636000e-05  1.58274000e-05  3.45280000e-01
  3.53600000e-01  6.17470200e+01  1.45993515e-24  9.78870106e-12
  1.09610943e-12  1.26736327e-04  9.67755102e-05  2.34049693e-01
  2.15523251e-18  7.56851082e-19  2.69818506e-12 -1.26803612e-14
 -2.57304163e-08 -8.23640000e-09 -1.09693061e-14 -2.10907755e-08
 -7.32942857e-09  1.09338776e-04  5.48661418e-13  6.17133396e-07
  2.74174135e-07 -4.58506694e-03 -4.11333878e-03]
[ 2.26678000e-12  4.23222000e-05  1.6469

/home/kkwakzii/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [209]:
def mean_cov(y_all, num_in_cycle, num_of_cycle, num_output):
    
    print("Y_all size: ", y_all.shape)
    
    y_mean_cov_num = int(2*num_output+((num_output)**2-(num_output))/2)
    # print(y_mean_cov_num)

    y_mean_cov = np.zeros((num_of_cycle, y_mean_cov_num))
    # print(y_mean_cov.shape)
    
    for i in range(num_of_cycle):
        #print(i)        
        #print(i*num_in_cycle, (i+1)*num_in_cycle)
        temp = y_all[i*num_in_cycle:(i+1)*num_in_cycle,:]
    
        mean_y = np.mean(temp, axis=0)
        cov_y = np.cov(temp.T)
        # print(cov_y)
        
        # mean
        y_mean_cov[i, :num_output] = mean_y
        # print(y_mean_cov)
        # diagonal
        y_mean_cov[i, num_output:num_output*2] = np.diagonal(cov_y)
        # print(y_mean_cov)
        
        # covariance
        cnt = num_output*2
        for j in range(1, num_output):
            y_mean_cov[i,cnt:cnt+j] = cov_y[j,:j]
            cnt += j
            
    return y_mean_cov
            
        #for k in range(y_mean_cov_num):
            # print(y_mean_cov[i,j], end=' ')
            
        #print("compare")
        #print(y_mean_cov[i,:num_output])
        #print(mean_y)
        #print()
        #print(y_mean_cov[i, num_output:])
        #print(cov_y)
            
            

        
                    

In [222]:
def normalize(x, y):
        
    print(x.shape, y.shape)
    for i in range(len(y)):
        print(y[i])
    x_mean = np.mean(x, axis=0, dtype=np.float32)
    x_std = np.std(x, axis=0, dtype=np.float32)
        
    y_mean = np.mean(y, axis=0, dtype=np.float32)
    
    y_std = np.std(y*1e+10, axis=0, dtype=np.float32)
    print(y_std)

    norm_x = ( x - x_mean ) / (x_std) 
    norm_y = ( y - y_mean ) / (y_std) * 1e+10
        
    return norm_x, norm_y, y_mean, y_std

In [183]:
x = np.array([[0, 2], [1, 1], [2, 0], [1, 3]]).T
cov = np.cov(x)
print(cov[1,0:1])

[-0.66666667]
